# import

In [1]:
import os
import sys
import json
import time
import uuid
from datetime import datetime
import subprocess
import numpy as np
import pandas as pd
from typing import Dict, List, Optional
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.models import resnet18
from torchvision.transforms import v2
from torch.utils.data import DataLoader, random_split
from timm.scheduler import CosineLRScheduler
import onnxruntime
from onnxruntime.quantization import (
   quantize_dynamic, quantize_static, QuantType, QuantFormat, CalibrationDataReader
)
from typing import Union
from sqlalchemy import create_engine, text
from sqlalchemy.orm import sessionmaker
from sqlalchemy import text
from sqlalchemy.orm import Session

# パス設定
sys.path.append("../configs")
sys.path.append('../db/')

import config as cf
from config import seed_everything

import src.db.models as models
import src.db.schemas as schemas
import src.db.cruds as cruds

# パス読み込み
RAW_MODELS_PATH = cf.RAW_MODELS_PATH[0]
PREPROCESSED_MODELS_PATH = cf.PREPROCESSED_MODELS_PATH[0]
QUANTIZED_MODELS_PATH = cf.QUANTIZED_MODELS_PATH[0]
MODELS_DB_PATH= cf.MODELS_DB_PATH[0]

# DB設定読み込み
with open('C:/Users/thyt/confidential_files/Postgresql/config.json', 'r', encoding='utf-8') as file:
    config = json.load(file)

class DBConfigurations:
    postgres_username = config["POSTGRES_USER"]
    postgres_password = config["POSTGRES_PASSWORD"]
    postgres_port = int(config["POSTGRES_PORT"])
    postgres_db = config["POSTGRES_DB"]
    postgres_server = config["POSTGRES_SERVER"]
    sql_alchemy_database_url = f"postgresql://{postgres_username}:{postgres_password}@{postgres_server}:{postgres_port}/{postgres_db}"

engine = create_engine(DBConfigurations.sql_alchemy_database_url, pool_recycle=3600, echo=False)
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)
db = SessionLocal()


c:\Users\thyt\envs\pytorchenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\thyt\envs\pytorchenv\lib\site-packages\pydantic\_internal\_config.py:321: UserWarning: Valid config keys have changed in V2:
* 'orm_mode' has been renamed to 'from_attributes'
  warnings.warn(message, UserWarning)
c:\Users\thyt\envs\pytorchenv\lib\site-packages\pydantic\_internal\_fields.py:149: UserWarning: Field "model_name" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
c:\Users\thyt\envs\pytorchenv\lib\site-packages\pydantic\_internal\_fields.py:149: UserWarning: Field "model_id" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespace

## CFG

In [2]:
class CFG:
    commit_hash="AAAAAA"
    project_id="207503"

    now = datetime.now()
    formatted_date = now.strftime("%Y%m%d_%H%M%S")

    file_prefix = f"{commit_hash}_{formatted_date}"
    num_workers=2
    n_epochs = 1
    device = "cuda" if torch.cuda.is_available() else "cpu"
    providers = ["CUDAExecutionProvider"]

seed_everything()

In [3]:
# データの水増しの設定
# データの水増し（データ拡張）のための設定
transform = transforms.Compose([
    transforms.ToTensor(),  # 画像をPyTorchテンソルに変換
    transforms.RandomResizedCrop(32, scale=(0.9, 1.0), antialias=False),  # 画像をランダムにリサイズし、32x32ピクセルにクロップ
    transforms.RandomRotation(degrees=(-15, 15)),  # 画像をランダムに-15度から15度の間で回転
    transforms.RandomHorizontalFlip(),  # 画像をランダムに水平方向に反転
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5]),  # 画像を正規化（平均0.5、標準偏差0.5で各チャンネルを正規化）
])

# CutMixの設定：画像の一部を別の画像で置き換える手法
cutmix = v2.CutMix(num_classes=10)
# MixUpの設定：2つの画像を重ね合わせて混合する手法
mixup = v2.MixUp(num_classes=10, alpha=0.2)
# CutMixとMixUpのうちどちらかをランダムに選択する
cutmix_or_mixup = v2.RandomChoice([cutmix, mixup])

# データのダウンロードとデータの読み込みとデータセットの作成
dataset = datasets.CIFAR10(root="./data", train=True, download=True, transform=transform)

# 8:2 に分割し、それぞれ訓練用と検証用とする
train_dataset, val_dataset = random_split(dataset, [0.8 , 0.2], generator=torch.Generator().manual_seed(42))

Files already downloaded and verified


# データローダーの作成

In [4]:
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=CFG.num_workers)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False, num_workers=CFG.num_workers)

# モデルの作成と損失関数の定義
model = resnet18(num_classes=10).to(CFG.device)
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)

# 最適化手法の定義と学習率スケジューラーの設定
optimizer = torch.optim.Adam(model.parameters())
scheduler = CosineLRScheduler(optimizer, t_initial=CFG.n_epochs, lr_min=1e-5, warmup_t=5, warmup_lr_init=1e-6, warmup_prefix=True)

Cosine annealing scheduler will have no effect on the learning rate since t_initial = t_mul = eta_mul = 1.


# モデルの学習と検証


In [5]:
for epoch in range(CFG.n_epochs):
    # 学習
    model.train()
    scheduler.step(epoch)  # 学習率更新
    for data, targets in train_loader:
        data, targets = data.to(CFG.device), targets.to(CFG.device)
        # CutUp or MixUp の適用
        data, targets = cutmix_or_mixup(data, targets)  # cutmix_or_mixup 関数は実装してください
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, targets)
        loss.backward()
        optimizer.step()

    # 検証
    model.eval()
    correct = 0
    total_loss = 0
    with torch.no_grad():
        for data, targets in val_loader:
            data, targets = data.to(CFG.device), targets.to(CFG.device)
            output = model(data)
            total_loss += criterion(output, targets)
            pred = output.argmax(axis=1)
            correct += (pred == targets).sum().item()
    # 検証結果の表示
    print(f"Epoch 数:{epoch + 1}, Accuracy (val): {correct / len(val_dataset)}, Loss: {total_loss / len(val_loader)}")


Epoch 数:1, Accuracy (val): 0.1196, Loss: 2.409409761428833


In [6]:
# print(data.shape)
# > torch.Size([16, 3, 32, 32])

## 3.2 量子化の準備

In [6]:
# まずはPyTorch形式からONNX形式に変換
x = torch.randn(1, 3, 32, 32, requires_grad=True)

# PyTorchモデルをONNX形式にエクスポート
torch.onnx.export(
    model.cpu().eval(),  # モデルを評価モードに設定,  # PyTorchモデル
    x,  # モデルへの入力
    f"{RAW_MODELS_PATH}/{CFG.file_prefix}_model.onnx", # ONNX 形式に変換されたモデルの保存先ファイルパス
    export_params=True,  # モデルの保存時にパラメータを含めるかどうか
    opset_version=12,  # 演算子のバージョン
    do_constant_folding=True,  # 定数入力を含む演算の一部を事前に計算された定数ノードに変換するかどうか
    input_names=["input"],  # モデルの入力に対する任意の名前
    output_names=["output"],  # モデルの出力に対する任意の名前
    dynamic_axes={"input": {0: "batch_size"},
                   "output": {0: "batch_size"}}  # 入力と出力の可変軸
)

# 次はモデルの前処理（モデルの最適化）を行います。
input_path = f"{RAW_MODELS_PATH}/{CFG.file_prefix}_model.onnx"
output_path = f"{PREPROCESSED_MODELS_PATH}/{CFG.file_prefix}_model_infer.onnx"

!python -m onnxruntime.quantization.preprocess --input {input_path} --output {output_path}

## 3.3 量子化の実行
- 動的量子化と静的量子化（QDQ とQOperator）を行う

### 動的量子化

In [8]:
# 次はモデルの前処理（モデルの最適化）を行います。

# wa_u8u8 は重みがu 8 、アクティベーションがu8 を表している
quantize_dynamic(f"{PREPROCESSED_MODELS_PATH}/{CFG.file_prefix}_model_infer.onnx",  # 前処理後のONNXモデルのパス
        f"{QUANTIZED_MODELS_PATH}/{CFG.file_prefix}_model_infer_dq_qop_wa_u8u8.onnx",  # 量子化後のONNXモデルの保存先パス
        weight_type=QuantType.QUInt8  # 重みの量子化方式を指定
        )

### QDQ での静的量子化

In [12]:
# キャリブレーション用データリーダーのクラス定義
class ImgDataReader(CalibrationDataReader):
    def __init__(self, imgs: np.ndarray) -> None:
        self.img_dicts = iter([{"input": img[None]} for img in imgs])
        [print(img[None]) for img in imgs]

    def get_next(self) -> Union[Dict[str, np.ndarray], None]:
        # get_next 関数は{"input": ndarray 形式} で返す、全て返し終わったらNone
        return next(self.img_dicts, None)

# ラベルごとに画像を仕分ける
label_imgs_dict = dict(zip(range(10), [[] for i in range(10)]))

for img, target in val_dataset:
    label_imgs_dict[target].append(img.numpy())

# 各ラベルを30 ずつとするデータを作成
calibration_data = []

for i in range(10):
    calibration_data.extend(label_imgs_dict[i][:30])

calibration_data = np.asarray(calibration_data)  # (300 , 3, 32, 32)

In [16]:
 for img in imgsで書く画像を取り出す
画像のshapeは(3, 32, 32)
img[None]で(1, 3, 32, 32)に

# キャリブレーション用データリーダーのクラス定義
class ImgDataReader(CalibrationDataReader):
    def __init__(self, imgs: np.ndarray) -> None:
        self.img_dicts = iter([{"input": img[None]} for img in imgs])
        # [print(img[None].shape) for img in imgs]
        [print(img.shape) for img in imgs]

    def get_next(self) -> Union[Dict[str, np.ndarray], None]:
        # get_next 関数は{"input": ndarray 形式} で返す、全て返し終わったらNone
        return next(self.img_dicts, None)

In [17]:
ImgDataReader(calibration_data)

(3, 32, 32)
(3, 32, 32)
(3, 32, 32)
(3, 32, 32)
(3, 32, 32)
(3, 32, 32)
(3, 32, 32)
(3, 32, 32)
(3, 32, 32)
(3, 32, 32)
(3, 32, 32)
(3, 32, 32)
(3, 32, 32)
(3, 32, 32)
(3, 32, 32)
(3, 32, 32)
(3, 32, 32)
(3, 32, 32)
(3, 32, 32)
(3, 32, 32)
(3, 32, 32)
(3, 32, 32)
(3, 32, 32)
(3, 32, 32)
(3, 32, 32)
(3, 32, 32)
(3, 32, 32)
(3, 32, 32)
(3, 32, 32)
(3, 32, 32)
(3, 32, 32)
(3, 32, 32)
(3, 32, 32)
(3, 32, 32)
(3, 32, 32)
(3, 32, 32)
(3, 32, 32)
(3, 32, 32)
(3, 32, 32)
(3, 32, 32)
(3, 32, 32)
(3, 32, 32)
(3, 32, 32)
(3, 32, 32)
(3, 32, 32)
(3, 32, 32)
(3, 32, 32)
(3, 32, 32)
(3, 32, 32)
(3, 32, 32)
(3, 32, 32)
(3, 32, 32)
(3, 32, 32)
(3, 32, 32)
(3, 32, 32)
(3, 32, 32)
(3, 32, 32)
(3, 32, 32)
(3, 32, 32)
(3, 32, 32)
(3, 32, 32)
(3, 32, 32)
(3, 32, 32)
(3, 32, 32)
(3, 32, 32)
(3, 32, 32)
(3, 32, 32)
(3, 32, 32)
(3, 32, 32)
(3, 32, 32)
(3, 32, 32)
(3, 32, 32)
(3, 32, 32)
(3, 32, 32)
(3, 32, 32)
(3, 32, 32)
(3, 32, 32)
(3, 32, 32)
(3, 32, 32)
(3, 32, 32)
(3, 32, 32)
(3, 32, 32)
(3, 32, 32)
(3, 

In [15]:
data_reader = ImgDataReader(calibration_data)
quant_format = QuantFormat.QDQ

# s8 は符号付き8bit 整数、u8 はデータ型が符号なし8bit 整数を表す
quantize_static(
    f"{PREPROCESSED_MODELS_PATH}/{CFG.file_prefix}_model_infer.onnx",  # 入力のONNXモデルのパス
    f"{QUANTIZED_MODELS_PATH}/{CFG.file_prefix}_model_infer_sq_qdq_wa_s8s8.onnx",  # 量子化後のONNXモデルの保存先パス
    data_reader,
    quant_format=quant_format,
    activation_type=QuantType.QUInt8
)

### QOperatorにおける静的量子化

In [16]:
data_reader = ImgDataReader(calibration_data)
quant_format = QuantFormat.QOperator

# s8 は符号付き8bit 整数、u8 はデータ型が符号なし8bit 整数を表す
quantize_static(
    f"{PREPROCESSED_MODELS_PATH}/{CFG.file_prefix}_model_infer.onnx",  # 入力のONNXモデルのパス
    f"{QUANTIZED_MODELS_PATH}/{CFG.file_prefix}_model_infer_sq_qop_wa_s8u8.onnx",  # 量子化後のONNXモデルの保存先パス
    data_reader,
    quant_format=quant_format,
    activation_type=QuantType.QUInt8
)

### モデルの保存

In [18]:
model = cruds.add_model(
    db=db, project_id=CFG.project_id,
    model_name=f"{CFG.file_prefix}_model.onnx",
    description="RAWモデル",
    commit = True,
    parent_model_id = None
    )
parent_model_id = model.model_id

model_infer = cruds.add_model(
    db=db, project_id=CFG.project_id,
    model_name=f"{CFG.file_prefix}_model_infer.onnx",
    description="前処理済みのモデル",
    commit = True,
    parent_model_id = parent_model_id
    )

model_infer_dq_qop_wa_u8u8 = cruds.add_model(
    db=db, project_id=CFG.project_id,
    model_name=f"{CFG.file_prefix}_model_infer_dq_qop_wa_u8u8.onnx",
    description="動的量子化済みモデル",
    commit = True,
    parent_model_id = parent_model_id
    )

model_infer_sq_qdq_wa_s8s8 = cruds.add_model(
    db=db, project_id=CFG.project_id,
    model_name=f"{CFG.file_prefix}_model_infer_sq_qdq_wa_s8s8.onnx",
    description="静的量子化（QDQ）済みモデル",
    commit = True,
    parent_model_id = parent_model_id
    )

model_infer_sq_qop_wa_s8u8 = cruds.add_model(
    db=db, project_id=CFG.project_id,
    model_name=f"{CFG.file_prefix}_model_infer_sq_qop_wa_s8u8.onnx",
    description="静的量子化（QOperator）済みモデル",
    commit = True,
    parent_model_id = parent_model_id
    )

model_id_model_infer = model_infer.model_id
model_id_model_infer_dq_qop_wa_u8u8 = model_infer_dq_qop_wa_u8u8.model_id
model_id_model_infer_sq_qdq_wa_s8s8 = model_infer_sq_qdq_wa_s8s8.model_id
model_id_model_infer_sq_qop_wa_s8u8 = model_infer_sq_qop_wa_s8u8.model_id

experiment_args = [
    {
        "model_path": f"{PREPROCESSED_MODELS_PATH}/{CFG.file_prefix}_model_infer.onnx",
        "model_version_id":"v0.1",
        "model_id":model_id_model_infer,
        "parameters":None,
        "training_dataset_path":"",
        "validation_dataset_path":"",
        "test_dataset_path":"",
        "artifact_file_paths":[f"{PREPROCESSED_MODELS_PATH}/{CFG.file_prefix}_model_infer.onnx"],
        "commit":True
    },
    {
        "model_path": f"{QUANTIZED_MODELS_PATH}/{CFG.file_prefix}_model_infer_dq_qop_wa_u8u8.onnx",  # 動的量子化済みモデルのファイルサイズ
        "model_version_id":"v0.1",
        "model_id":model_id_model_infer_dq_qop_wa_u8u8,
        "parameters":None,
        "training_dataset_path":"",
        "validation_dataset_path":"",
        "test_dataset_path":"",
        "artifact_file_paths":[f"{QUANTIZED_MODELS_PATH}/{CFG.file_prefix}_model_infer_dq_qop_wa_u8u8.onnx"],
        "commit":True
    },
    {
        "model_path": f"{QUANTIZED_MODELS_PATH}/{CFG.file_prefix}_model_infer_sq_qdq_wa_s8s8.onnx", # 静的量子化（QDQ）済みモデルのファイルサイズ
        "model_version_id":"v0.1",
        "model_id":model_id_model_infer_sq_qdq_wa_s8s8,
        "parameters":None,
        "training_dataset_path":"",
        "validation_dataset_path":"",
        "test_dataset_path":"",
        "artifact_file_paths":[f"{QUANTIZED_MODELS_PATH}/{CFG.file_prefix}_model_infer_sq_qdq_wa_s8s8.onnx"],
        "commit":True
    },
    {
        "model_path": f"{QUANTIZED_MODELS_PATH}/{CFG.file_prefix}_model_infer_sq_qop_wa_s8u8.onnx", # 静的量子化（QOperator）済みモデルのファイルサイズ
        "model_version_id":"v0.1",
        "model_id":model_id_model_infer_sq_qop_wa_s8u8,
        "parameters":None,
        "training_dataset_path":"",
        "validation_dataset_path":"",
        "test_dataset_path":"",
        "artifact_file_paths":[f"{QUANTIZED_MODELS_PATH}/{CFG.file_prefix}_model_infer_sq_qop_wa_s8u8.onnx"],
        "commit":True
    }
]

## 3.5 推論の実行

In [19]:
"""input_path には前処理済みモデル、動的量子化済みモデル、静的量子化済みモデル（QOperator）
ファイルパスをそれぞれ与えて以下の推論を実行します。
推論はGoogleのColaboratoryのCPU環境（2.20GHzのXeonで拡張命令セット
としてAVX2 を持つ）にて実行しました。"""
experiment_id_list = []

for experiment in experiment_args:
    input_path = experiment['model_path']

    session_fp32 = onnxruntime.InferenceSession(input_path, providers=CFG.providers)

    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5]),
    ])

    test_dataset = datasets.CIFAR10(root="./data", train=False, download=True, transform=transform)
    test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=CFG.num_workers)

    # 推論と時間計測
    correct = 0
    start_time = time.perf_counter()

    for data, targets in test_loader:
        data, targets = data.numpy(), targets.numpy()
        output = session_fp32.run([], {"input": data})[0]  # 推論の実行
        pred = output.argmax(axis=1)
        correct += (pred == targets).sum()

    end_time = time.perf_counter()

    evaluations ={
        "モデルの条件": input_path.split('/')[-1],
        "ファイルサイズ/MB":(str(os.path.getsize(f"{input_path}") / (1000**2))+ " MB"),  ## 3.4 ファイルサイズの計算
        "正解率/%":(correct / len(test_dataset)),
        "推論時間/s":(end_time - start_time)
        }

    result = cruds.add_experiment(
        db=db,
        model_version_id=experiment["model_version_id"],
        model_id=experiment["model_id"],
        parameters=experiment["parameters"],
        training_dataset_path=experiment["training_dataset_path"],
        validation_dataset_path=experiment["validation_dataset_path"],
        test_dataset_path=experiment["test_dataset_path"],
        evaluations=evaluations,  # 評価は元のコードから取得
        artifact_file_paths=experiment["artifact_file_paths"],
        commit=experiment["commit"]
    )
    experiment_id_list.append(result.experiment_id)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


## 3.6 精度・推論速度の可視化

In [20]:
# 空のリストを初期化
tmp = []

# experiment_id_listの各experiment_idに対して以下の処理を実行
for experiment_id in experiment_id_list:
    # model_idを取得（ここは使われていないようです）
    model_id = experiment["model_id"],

    # experiment_idに基づいて実験の詳細を取得し、tmpリストに追加
    tmp.append(
        [cruds.select_experiment_by_id(db, experiment_id).experiment_id,
         cruds.select_experiment_by_id(db, experiment_id).model_id,
         cruds.select_experiment_by_id(db, experiment_id).model_version_id,
         cruds.select_experiment_by_id(db, experiment_id).parameters,
         cruds.select_experiment_by_id(db, experiment_id).training_dataset_path,
         cruds.select_experiment_by_id(db, experiment_id).validation_dataset_path,
         cruds.select_experiment_by_id(db, experiment_id).test_dataset_path,
         cruds.select_experiment_by_id(db, experiment_id).evaluations,
         cruds.select_experiment_by_id(db, experiment_id).artifact_file_paths,
         cruds.select_experiment_by_id(db, experiment_id).created_datetime]
    )

# tmpを使用してデータフレームを作成し、列名を設定
experiment_results = pd.DataFrame(tmp, columns=["experiment_id", "model_id", "model_version_id", "parameters",
                                                "training_dataset_path", "validation_dataset_path",
                                                "test_dataset_path", "evaluations", "artifact_file_paths",
                                                "created_datetime"])

# experiment_resultsデータフレームを表示
experiment_results

# 評価結果を格納するための空のリストを初期化
data_list = []

# experiment_resultsの各行に対して以下の処理を実行
for _, row in experiment_results.iterrows():
    # 評価結果をdata_dictに格納
    data_dict = row['evaluations']

    # data_listにdata_dictを追加
    data_list.append(data_dict)

# data_listを使用して評価結果のデータフレームを作成
pd.DataFrame(data_list)


,モデルの条件,ファイルサイズ/MB,正解率/%,推論時間/s
0,AAAAAA_20231125_180833_model_infer.onnx,44.720849 MB,0.1285,7.607179
1,AAAAAA_20231125_180833_model_infer_dq_qop_wa_u...,11.234565 MB,0.1274,26.624171
2,AAAAAA_20231125_180833_model_infer_sq_qdq_wa_s...,11.236189 MB,0.1280,7.853049
3,AAAAAA_20231125_180833_model_infer_sq_qop_wa_s...,11.217224 MB,0.1290,7.213629


1. ファイルサイズの比較：
   - 最初のモデル（`model_infer.onnx`）は約44.72MBで、他のモデルに比べて大きいです。
   - 残りの3つのモデルは約11.23MBで、かなりサイズが小さくなっています。

2. 正解率の比較：
   - すべてのモデルの正解率は似ており、約0.128（12.8%）です。
   - 最高の正解率は`model_infer_dq_qop_wa_u8u8.onnx`の0.1297（12.97%）で、最低は`model_infer_sq_qdq_wa_s8s8.onnx`の0.1275（12.75%）です。
   - これらの差は非常に小さいため、モデル間の正解率に大きな差はないと言えます。

3. 推論時間の比較：
   - 最初のモデル（`model_infer.onnx`）と`model_infer_sq_qdq_wa_s8s8.onnx`の推論時間はほぼ同じで、約8秒です。
   - `model_infer_sq_qop_wa_s8u8.onnx`は最も推論時間が短く、約7.12秒です。
   - `model_infer_dq_qop_wa_u8u8.onnx`は、推論に約24.41秒とかなり長い時間がかかります。

- 総合的な考察：
   - ファイルサイズを大幅に削減しても、正解率には大きな影響が見られません。
   - ただし、`model_infer_dq_qop_wa_u8u8.onnx`は他のモデルに比べて推論時間が大幅に長くなっています。これは、サイズ削減のための最適化が推論速度に影響を与えている可能性があります。
   - `model_infer_sq_qop_wa_s8u8.onnx`は、ファイルサイズを大幅に削減しつつ、推論時間を短縮しているため、最適なバランスを提供している可能性があります。

In [21]:
import winsound

# ビープ音を再生する
winsound.Beep(1000, 500)  # 周波数1000Hzで500ミリ秒のビープ音

# WAVファイルを再生する
winsound.PlaySound("sound.wav", winsound.SND_FILENAME)


## DB操作

In [22]:
sql_query = text("SELECT * FROM projects")
results = db.execute(sql_query)
data = [result for result in results]
df_projects = pd.DataFrame(data)

sql_query = text("SELECT * FROM models")
results = db.execute(sql_query)
data = [result for result in results]
df_models = pd.DataFrame(data)

sql_query = text("SELECT * FROM experiments")
results = db.execute(sql_query)
data = [result for result in results]
df_experiments = pd.DataFrame(data)

# df_projectsとdf_modelsをproject_idでleft join
df_joined = pd.merge(df_projects, df_models, on="project_id", how="left")

# 結合後のデータフレームdf_joinedとdf_experimentsをmodel_idでleft join
final_df = pd.merge(df_joined, df_experiments, on="model_id", how="left")

tmpdf = pd.DataFrame()
# evaluations列をイテレーション
for index, row in final_df.iterrows():
    tmpdf = pd.concat([tmpdf,pd.DataFrame([row['evaluations']])],axis=0)

tmpdf = tmpdf.drop(columns=[0])
tmpdf

,モデルの条件,ファイルサイズ/MB,正解率/%,推論時間/s
0,NaN,NaN,NaN,NaN
0,AAAAAA_20231125_011819_model_infer.onnx,44.720849 MB,0.8137,7.883786
0,AAAAAA_20231125_011819_model_infer_dq_qop_wa_u...,11.234564 MB,0.8131,24.908645
0,AAAAAA_20231125_011819_model_infer_sq_qdq_wa_s...,11.236187 MB,0.8133,7.425493
0,AAAAAA_20231125_011819_model_infer_sq_qop_wa_s...,11.217222 MB,0.8132,6.841749
0,NaN,NaN,NaN,NaN
0,AAAAAA_20231125_180833_model_infer.onnx,44.720849 MB,0.1285,7.607179
0,AAAAAA_20231125_180833_model_infer_dq_qop_wa_u...,11.234565 MB,0.1274,26.624171
0,AAAAAA_20231125_180833_model_infer_sq_qdq_wa_s...,11.236189 MB,0.1280,7.853049
0,AAAAAA_20231125_180833_model_infer_sq_qop_wa_s...,11.217224 MB,0.1290,7.213629
